In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.entities import Workspace, AmlCompute, Datastore
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils
import pandas as pd
import datetime
import sys
from pathlib import Path


In [66]:
subscription_id = '31bc5bb0-1143-4d0c-ba02-570ecc764364'
resource_group = 'Face_Attribute_Recognition'
workspace_name = 'test_ws'
cluster_name = 'cpu-cluster'

In [67]:
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [ ]:
ws_demo = Workspace(
    name = 'test_ws',
    location= 'germanywestcentral',
    display_name = 'Test workspace',
    hbi_workspace = False,
    description = 'A simple test workspace',
    tags = dict(purpuse='demo')
)

ws_demo = ml_client.workspaces.begin_create(ws_demo).result()

In [68]:
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace_name)
ws = ml_client._workspaces.get(name = workspace_name)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [61]:
local_folder_path = '../data/celeba/test_dataset/'
v1 = 'initial'
name = 'test_dataset'


my_data = Data(
    name=name,
    version=v1,
    description='Celeba csv folder',
    path=local_folder_path,
    type=AssetTypes.URI_FOLDER,
)

try:
    data_asset = ml_client.data.get(name=name, version=v1)
    print(f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}")
except:
    data_asset = ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

Uploading test_dataset (43.78 MBs): 100%|██████████| 43782772/43782772 [00:15<00:00, 2868184.56it/s]




Data asset created. Name: test_dataset, version: initial


In [70]:
data_asset = ml_client.data.get(name='celeba', version='initial')
data_asset.path

'azureml://subscriptions/31bc5bb0-1143-4d0c-ba02-570ecc764364/resourcegroups/Face_Attribute_Recognition/workspaces/test_ws/datastores/workspaceblobstore/paths/LocalUpload/1d6e11d9c342192c4d9259914c8f4ee7/celeba/'

In [4]:
try:
    cpu_cluster = ml_client.compute.get(cluster_name)
    print(f'Compute target named "{cluster_name}" already exists!')

except ResourceNotFoundError:
    print('Creating a new cpu compute target...')
    cpu_cluster = AmlCompute(
        name = cluster_name,
        size = 'STANDARD_D2_V3',
        min_instances = 0,
        max_instances = 1,
        idle_time_before_scale_down = 180,
        tier = 'Dedicated'
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    ml_client.compute.begin_create_or_update(cpu_cluster)




Compute target named "cpu-cluster" already exists!


In [6]:
ml_client.compute.begin_delete(cluster_name)